In [1]:
import pandas as pd
import requests
import json
import time
import random
from pathlib import Path

In [2]:
def load_config():
    config_path = Path('config_extract_support_tables.json')
    if config_path.exists():
        with open(config_path) as f:
            return json.load(f)
    else:
        raise FileNotFoundError("Config file not found. Ensure a config.json is present and correctly configured.")

In [3]:
def get_brand_model_names(car_brand_ids):
    config = load_config()
    url_template = config["url_template"]
    headers = config["headers"]

    for brand in car_brand_ids:
        # Format URL with the brand id
        url = url_template.format(brand=brand)
        
        response = requests.get(url, headers=headers)

        if response.status_code == 200:  # Check if the request was successful
            print(f"Brand id: {brand} | Status code: {response.status_code}\n")
            with open(f'01_brand_models_json_files/brandId_{brand}.json', 'w') as fp:
                json.dump(response.json()['items'], fp)
        else:
            print(f"Brand id {brand} had an issue. Status code: {response.status_code}")

        delay = random.uniform(3, 5)
        print(f"Waiting {delay:.2f} seconds before the next request.")
        time.sleep(delay)

    return "All requests completed"

In [4]:
makers_df = pd.read_csv('./01_brand_models_json_files/brands.csv')

In [ ]:
# get_brand_model_names(car_brand_ids=makers_df['id'].values)

In [52]:
for brand in makers_df['id'].values:

    if brand == 1:
        all_brands_df = pd.read_json(f'./01_brand_models_json_files/brandId_{brand}.json')
        all_brands_df['brandId'] = brand
    else:
        temp_df = pd.read_json(f'./01_brand_models_json_files/brandId_{brand}.json')
        temp_df['brandId'] = brand
        all_brands_df = pd.concat([all_brands_df, temp_df])     

In [55]:
all_brands_df['modelId'] = all_brands_df['modelId'].astype(int)

In [73]:
df_brand_models = all_brands_df.merge(makers_df, left_on='brandId', right_on='id', how='left')

In [86]:
df_brand_models = (df_brand_models
    # .drop('id', axis=1)
    .rename({'label':'brand', 'name':'model', 'brandId':'brand_id', 'modelId':'model_id'}, axis=1))

In [79]:
df_brand_models['model'] = df_brand_models['model'].str.lower()

In [80]:
df_brand_models['brand'] = df_brand_models['brand'].str.lower()

In [88]:
df_brand_models.to_csv('../02_load/brand_models.csv', index=False) 

In [6]:
df_brand_models = pd.read_csv('../02_load/brand_models.csv')